##### PDF RAG LLM with Mistral and Langchain

##### Poojitha Venkatram, Shruti Badrinarayanan

In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  U

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
# ! pip install transformers -U

### Get data



In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import PyPDF2

# Open the PDF file
with open("/content/drive/MyDrive/Sample Data.pdf", "rb") as file:
    pdf = PyPDF2.PdfReader(file)

    # Initialize a variable to hold all text
    full_text = ""

    # Iterate through each page and extract text
    for page in pdf.pages:
        full_text += page.extract_text()

# Printing the extracted text first 500 characters to check the extraction
print(full_text[:500])

   
 
   
  
 
 
Gap Inc. Reports First  Quarter Fiscal 2023 Results   
 
SAN FRANCISCO – May 25 , 202 3 – Gap Inc . (NYSE: GPS), a portfolio of purpose -led, billion -dollar lifestyle  
brands including Old Navy, Gap, Banana Republic, and Athleta, and the largest specialty apparel 
company in the U.S ., today reported financial results for its first  quarter ended April 29, 202 3.  
 “We continue to take the necessary actions to drive critical change at Gap Inc., ultimately getting us 
back on 


In [ ]:
len(full_text)

33762

## Split document into chunks



In [ ]:
chunk_size = 2048
chunks = [full_text[i:i + chunk_size] for i in range(0, len(full_text), chunk_size)]

In [ ]:
len(chunks)

17

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_text_embedding(text):
    # Tokenize the input text and convert to input tensors
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate embeddings
    with torch.no_grad():
        model_output = model(**tokens)

    # You might want to take the mean of the last hidden state as the embedding
    embeddings = model_output.last_hidden_state.mean(dim=1).numpy()
    return embeddings

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
import numpy as np
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
text_embeddings.shape

(17, 1, 384)

In [ ]:
text_embeddings

array([[[-0.10994791, -0.08910654,  0.0248667 , ..., -0.29204887,
         -0.03579939,  0.11007948]],

       [[ 0.03193063,  0.04148663, -0.02880092, ..., -0.18756476,
          0.01853206,  0.03418391]],

       [[ 0.00222948, -0.09604086,  0.04064266, ..., -0.27956256,
         -0.05059518,  0.12926713]],

       ...,

       [[-0.05055098,  0.021625  ,  0.02087245, ..., -0.25107798,
         -0.07013686, -0.01034945]],

       [[ 0.1440477 , -0.17392927,  0.02351383, ..., -0.37629646,
         -0.12263596,  0.08559659]],

       [[-0.00593987, -0.1442028 ,  0.03567518, ..., -0.31001282,
         -0.148765  ,  0.10683215]]], dtype=float32)

In [ ]:
import numpy as np

def get_text_embedding(text):
    # This should return a real embedding based on your model
    return np.random.rand(384)

# Example list of texts
texts = ["text1", "text2", "text3"]

# Generate embeddings for each text
text_embeddings = np.array([get_text_embedding(text) for text in texts])

print("Generated embeddings shape:", text_embeddings.shape)

Generated embeddings shape: (3, 384)


In [ ]:
import faiss

# Assuming text_embeddings is your array with shape (3, 384)
d = text_embeddings.shape[1]  # Dimension of the embeddings (should be 384)

# Creating the FAISS index
index = faiss.IndexFlatL2(d)

# Adding the embeddings to the index
index.add(text_embeddings)

### Load into a vector database



In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)



### Create embeddings for a question
Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.


In [ ]:
question = "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 384)

In [ ]:
question_embeddings

array([[4.74735852e-01, 9.86006073e-01, 9.39267152e-01, 5.28079789e-01,
        1.82000296e-01, 4.77332690e-01, 3.21752519e-01, 8.58625153e-01,
        7.23801756e-01, 6.56963369e-01, 1.86732405e-01, 3.50643195e-01,
        2.10537342e-01, 2.18585106e-01, 9.88117819e-01, 6.70459147e-01,
        3.22496110e-01, 4.92877638e-01, 5.75467510e-01, 4.28339286e-01,
        4.61128550e-01, 6.74781359e-01, 7.37751841e-01, 4.39419694e-01,
        1.67446400e-01, 2.93328194e-01, 8.58468231e-01, 9.46904957e-01,
        1.89814506e-01, 3.93622632e-01, 1.23856879e-01, 7.73022097e-01,
        6.59241137e-02, 8.72253768e-01, 8.57506484e-01, 3.41950277e-01,
        7.79979270e-01, 5.93413198e-01, 2.42920055e-01, 2.10552704e-01,
        3.78572250e-01, 8.00631458e-01, 4.64897940e-01, 5.43186938e-01,
        9.68909274e-01, 9.81724186e-01, 4.06583366e-01, 7.51619648e-01,
        6.03184862e-01, 5.60323504e-01, 4.77019549e-01, 6.33199298e-01,
        5.55572784e-01, 9.09454290e-01, 2.80677477e-01, 8.838692



### Retrieve similar chunks from the vector database
We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database. Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.


In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[0 2]]


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['   \n \n   \n  \n \n \nGap Inc. Reports First  Quarter Fiscal 2023 Results   \n \nSAN FRANCISCO – May 25 , 202 3 – Gap Inc . (NYSE: GPS), a portfolio of purpose -led, billion -dollar lifestyle  \nbrands including Old Navy, Gap, Banana Republic, and Athleta, and the largest specialty apparel \ncompany in the U.S ., today reported financial results for its first  quarter ended April 29, 202 3.  \n “We continue to take the necessary actions to drive critical change at Gap Inc., ultimately getting us \nback on a path toward delivering consistent results long -term,” said Bob Martin, Executive Chairman \nand Interim CEO, Gap Inc. “While the macro and consum er environment remain uncertain, Q1 \nunderscores our ability to deliver improvements to the business including share gains at Old Navy and \nGap Brand, adjusted operating margin expansion, reduction in inventory, and strength in our balance \nsheet. The need fo r lasting change is permeating the organization and I want to express my g

In [ ]:
import pandas as pd

# Assuming retrieved_chunk is your list of text chunks corresponding to the nearest neighbors
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

# Creating a DataFrame
df = pd.DataFrame(retrieved_chunk, columns=['Retrieved Chunks'])

# Display the DataFrame
print(df)

                                    Retrieved Chunks
0     \n \n   \n  \n \n \nGap Inc. Reports First ...
1  re, totaling $ 55 million . Board of Directors...


In [ ]:
import pandas as pd

# Set option to increase width and max_colwidth
pd.set_option('display.max_colwidth', None)  # For pandas versions >= 1.0
pd.set_option('display.width', 1000)  # Adjust as necessary for your display

df_full_text = pd.DataFrame([full_text], columns=['Full Text'])
print(df_full_text)

In [ ]:
import pandas as pd

# Assuming df_full_text is already filled with the PDF text data
context = " ".join(df_full_text['Full Text'].tolist())


  
### Combine context and question in a prompt and generate response

Finally, we can present the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [ ]:
import pandas as pd
import numpy as np
import faiss

context = " ".join(df_full_text['Full Text'].tolist())

In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = False,
    model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    temperature=0.3,
    max_new_tokens=512,
    top_k=50,
    top_p=0.95,
    verbose=False,
    n_ctx=4096
)

In [ ]:
question = "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?"

prompt = [f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""]

# Call the generate method with the prompt as a list
answer = llm.generate(prompt)

# Print the generated answer
print("Generated Answer:", answer)

Generated Answer: generations=[[Generation(text='Based on the provided context information, it appears that Gap Inc. reported financial results for its first quarter ended April 29, 2023. The context information includes various financial metrics such as net sales, comparable sales, gross margin, operating income, and net income. However, the context information does not explicitly provide the operating profit or loss amount for Gap Inc. in the first quarter of fiscal 2023. Therefore, it is not possible to definitively answer the query with the given context information alone.')]] llm_output=None run=[RunInfo(run_id=UUID('bde78460-d9cd-45dd-8675-8705e61d9097'))]


In [ ]:
question = "What were the key results from each of Gap Inc.'s brands (Old Navy, Gap, Banana Republic, and Athleta)?"

prompt = [f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""]

# Call the generate method with the prompt as a list
answer = llm.generate(prompt)

# Print the generated answer
print("Generated Answer:", answer)

Generated Answer: generations=[[Generation(text='1. Old Navy: Net sales were $1.8 billion, down 1% compared to last year. Comparable sales were down 1%.\n2. Gap: Net sales were $692 million, down 13% compared to last year. Comparable sales were up 1%.\n3. Banana Republic: Net sales were $432 million, down 10% on top of 24% growth last year. Comparable sales were down 8%.\n4. Athleta: Net sales were $321 million, down 11% compared to last year. Comparable sales were down 13%.')]] llm_output=None run=[RunInfo(run_id=UUID('4b1ac40e-c132-45d3-bbac-40525d61a583'))]


### Mistral Answers

In [ ]:
questions = [
    "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?",
    "What is the report quarter and when did it end?",
    "What are the results for the reporting quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, op margin, net income, and EPS. Include both GAAP and non-GAAP numbers. Please also include quarter-over-quarter changes.",
    "If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, what is the cost for a household that generates 30 pounds per week?",
    "What's the operating margin for the current quarter?",
    "Identify potential risks associated with EPR implementation, such as regulatory changes, market volatility, and technological disruptions. Develop a risk assessment framework that assigns probabilities and impacts to each identified risk factor. Propose risk mitigation strategies for minimizing the adverse effects of these risks on EPR programs and stakeholder interests.",
    "What were Gap Inc's net sales for the first quarter of fiscal 2023 and how did they compare to the previous year?",
    "How did merchandise margin change from the previous year, and what were the drivers of this change?",
    "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?",
    "What were the key results from each of Gap Inc.'s brands (Old Navy, Gap, Banana Republic, and Athleta)?",
    "What is the company's outlook on net sales for the second quarter and the full fiscal year 2023?",
    "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?",
    "How does the company expect gross margin to change in Q2 and the full year compared to 2022?",
    "Did Gap Inc. pay a dividend in the first quarter, and what is their dividend strategy going forward?",
    "What are the key actions Gap Inc. is taking to execute its plan?"
]

In [ ]:
import pandas as pd

# Define a list to store all the answers
all_answers = []

# Iterate through each question
for question in questions:
    # Construct the prompt for the question
    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    # Call the generate method with the prompt as a list
    answer = llm.generate([prompt])
    # Append the generated answer to the list of all answers
    all_answers.append(answer)

# Print all the generated answers
for i, answer in enumerate(all_answers):
    print(f"Question {i+1}: {questions[i]}")
    print(f"Generated Answer: {answer}")
    print()

Question 1: A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?
Generated Answer: generations=[[Generation(text='500 tons x 80% = 400 tons\n    The company should aim to recover 400 tons of plastic packaging annually to meet the EPR requirement.')]] llm_output=None run=[RunInfo(run_id=UUID('870d15a9-ba2f-4a5c-ac2d-1de2fb8efdc6'))]

Question 2: What is the report quarter and when did it end?
Generated Answer: generations=[[Generation(text='1. The report quarter refers to the first quarter of a fiscal year.\n    2. Based on the context information provided, the report quarter is the first quarter of Fiscal 2023.\n    3. The context information also states that the first quarter of Fiscal 2023 ended on April 29, 2023.')]] llm_output=None run=[RunInfo(run_id=UUID('8198e219-c006-436

In [ ]:
import pandas as pd

# Define a dictionary to store questions and answers
data = {'Question': questions, 'Answer': all_answers}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('Mistral Output.csv', index=False)

print("CSV file saved successfully.")

CSV file saved successfully.


### Setup Questions, Ground Truths, Rag Answer Lists

In [ ]:
questions = [
    "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?",
    "What is the report quarter and when did it end?",
    "What are the results for the reporting quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, op margin, net income, and EPS. Include both GAAP and non-GAAP numbers. Please also include quarter-over-quarter changes.",
    "If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, what is the cost for a household that generates 30 pounds per week?",
    "What's the OP margin for the current quarter?",
    "Identify potential risks associated with EPR implementation, such as regulatory changes, market volatility, and technological disruptions. Develop a risk assessment framework that assigns probabilities and impacts to each identified risk factor. Propose risk mitigation strategies for minimizing the adverse effects of these risks on EPR programs and stakeholder interests.",
    "What were Gap Inc's net sales for the first quarter of fiscal 2023 and how did they compare to the previous year?",
    "How did merchandise margin change from the previous year, and what were the drivers of this change?",
    "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?",
    "What were the key results from each of Gap Inc.'s brands (Old Navy, Gap, Banana Republic, and Athleta)?",
    "What is the company's outlook on net sales for the second quarter and the full fiscal year 2023?",
    "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?",
    "How does the company expect gross margin to change in Q2 and the full year compared to 2022?",
    "Did Gap Inc. pay a dividend in the first quarter, and what is their dividend strategy going forward?",
    "What are the key actions Gap Inc. is taking to execute its plan?"
]

ground_truths = [
    "EPR Recovery Target Calculation:\n To meet the recovery target of 80%, the company should aim to recover 80% of 500 tons, which is:\n Recovery Target=80%×500 tons = 0.80×500 tons=400 tons\n So, the company should aim to recover 400 tons of plastic packaging each year.",  # Q1
    "The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023",  # Q2
    "The Results for the reporting quarter are:\n 1. Revenue: GAAP revenue was $1,214 million (up 3% YoY and 2% QoQ), while non-GAAP revenue stood at $1,200 million (up 3% YoY and 2% QoQ).\n2. Gross Margin: GAAP gross margin was 37.1% (down 0.3% YoY and flat QoQ), non-GAAP was 36.6% (down 0.4% YoY and flat QoQ).\n 3. Operating Expenses (Opex): GAAP operating expenses were $714 million (up 2% YoY and 1% QoQ), non-GAAP expenses were $687 million (up 2% YoY and 1% QoQ).\n 4. Operating Margin: GAAP operating margin came in at 13.1% (down 0.2% YoY and flat QoQ), and non-GAAP was 12.5% (down 0.3% YoY and flat QoQ).\n 5. Net Income: GAAP net income was $18 million (down 62% YoY and 85% QoQ), non-GAAP net income was $3 million (down 67% YoY and 88% QoQ).\n 6. EPS: GAAP EPS was $0.05 per share (down 62% YoY and 85% QoQ), non-GAAP EPS was $0.01 per share (down 67% YoY and 88% QoQ).",  # Q3
    "A household generating 30 pounds of waste per week would incur a cost of $1.50 weekly under a pay-as-you-throw program charging $0.05 per pound.",  # Q4
    "Reported operating margin was -0.3%. Adjusted operating margin was 0.5%.",  # Q5
    "Potential Risks in EPR Implementation: \n * Regulatory Changes: High probability (0.7), high impact (0.8) \n * Market Volatility: Moderate probability (0.6), moderate impact (0.5) \n * Technological Disruptions: Low probability (0.4), low impact (0.3) \n Risk Mitigation Strategies: \n * Regulatory Changes: Proactively monitor changes, maintain open dialogue, design flexible programs. \n * Market Volatility: Offer diverse EPR portfolio, implement risk management. \n * Technological Disruptions: Prioritize R&D, create contingency plans. \n Key Takeaway: Risk assessment and proactive mitigation are crucial for successful EPR programs.",  # Q6
    "Gap Inc's net sales for Q1 fiscal 2023 were $3.28 billion, down 6% from last year which is inclusive of an estimated 1-point foreign exchange headwind and 2 percentage points of negative impact from the sale of Gap China.",  # Q7
    "Merchandise margin increased 600 basis points versus last year, or 610 basis points on an adjusted basis, due to lower air freight expense and improved promotional activity in the quarter, partially offset by inflationary cost headwinds.",  # Q8
    "Reported operating loss of $10 million. Adjusted operating income was $18 million.",  # Q9
    "The key results for Gap Inc.'s brands in the first quarter are the following:\n* Old Navy: $1.8 billion in revenue, sales down 1%.\n* Gap: $692 million in revenue, sales down 13%.\n* Banana Republic: $432 million in revenue, sales down 10%.\n* Athleta: $321 million in revenue, sales down 11%.",  # Q10
    "Second quarter net sales expected to decrease mid to high-single digits. Fiscal 2023 net sales expected to decrease low to mid-single digits.",  # Q11
    "Gap Inc. had $1.2 billion in cash and cash equivalents at the end of the quarter.",  # Q12
    "Gross margin is expected to expand in Q2 and the full year compared to the prior year.",  # Q13
    "Paid a dividend of $0.15 per share in the first quarter. The Board approved the same for the second quarter.",  # Q14
    "Key actions include driving critical change for consistent long-term results, focusing on customer experience, reducing inventory, and strengthening the balance sheet."  # Q15
]

In [ ]:
# Check the lengths of all three lists
print(f"Length of 'questions': {len(questions)}")
print(f"Length of 'all_answers': {len(all_answers)}")
print(f"Length of 'ground_truths': {len(ground_truths)}")

Length of 'questions': 15
Length of 'all_answers': 15
Length of 'ground_truths': 15


In [ ]:
import pandas as pd
import re

df = pd.DataFrame({
    'Question': questions,
    'Generated Answer': all_answers,
    'Ground Truth': ground_truths
})

# Save the DataFrame to a CSV file
df.to_csv('generated_answers_with_ground_truths.csv', index=False)
print("CSV File saved successfully.")

# Display the DataFrame
df

CSV File saved successfully.


,Question,Generated Answer,Ground Truth
0,A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?,generations=[[Generation(text='500 tons x 80% = 400 tons\n The company should aim to recover 400 tons of plastic packaging annually to meet the EPR requirement.')]] llm_output=None run=[RunInfo(run_id=UUID('870d15a9-ba2f-4a5c-ac2d-1de2fb8efdc6'))],"EPR Recovery Target Calculation:\n To meet the recovery target of 80%, the company should aim to recover 80% of 500 tons, which is:\n Recovery Target=80%×500 tons = 0.80×500 tons=400 tons\n So, the company should aim to recover 400 tons of plastic packaging each year."
1,What is the report quarter and when did it end?,"generations=[[Generation(text='1. The report quarter refers to the first quarter of a fiscal year.\n 2. Based on the context information provided, the report quarter is the first quarter of Fiscal 2023.\n 3. The context information also states that the first quarter of Fiscal 2023 ended on April 29, 2023.')]] llm_output=None run=[RunInfo(run_id=UUID('8198e219-c006-4364-a611-7c2c41108405'))]","The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"
2,"What are the results for the reporting quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, op margin, net income, and EPS. Include both GAAP and non-GAAP numbers. Please also include quarter-over-quarter changes.","generations=[[Generation(text='---------------------\n • Revenue: Net sales for the reporting quarter were $3.28 billion , down 6% compared to last year.\n - Quarter-over-quarter change: Down 6%.\n • Gross margin: For the reporting quarter, gross margin was 37.9%, a decrease of 1.4 percentage points compared to last year.\n - Quarter-over-quarter change: No significant change.\n • Operating expenses (opex): Opex for the reporting quarter was $1.26 billion , down 5% compared to last year.\n - Quarter-over-quarter change: Down 5%.\n • Operating margin (op margin): Operating margin for the reporting quarter was 10.8%, a decrease of 1.7 percentage points compared to last year.\n - Quarter-over-quarter change: Down 1.7 percentage points.\n • Net income: For the reporting quarter, net income was $235 million , down 28% compared to last year.\n - Quarter-over-quarter change: Down 28%.\n • Earnings per share')]] llm_output=None run=[RunInfo(run_id=UUID('45f870a9-8f14-424d-b308-23d6c7ff5453'))]","The Results for the reporting quarter are:\n 1. Revenue: GAAP revenue was $1,214 million (up 3% YoY and 2% QoQ), while non-GAAP revenue stood at $1,200 million (up 3% YoY and 2% QoQ).\n2. Gross Margin: GAAP gross margin was 37.1% (down 0.3% YoY and flat QoQ), non-GAAP was 36.6% (down 0.4% YoY and flat QoQ).\n 3. Operating Expenses (Opex): GAAP operating expenses were $714 million (up 2% YoY and 1% QoQ), non-GAAP expenses were $687 million (up 2% YoY and 1% QoQ).\n 4. Operating Margin: GAAP operating margin came in at 13.1% (down 0.2% YoY and flat QoQ), and non-GAAP was 12.5% (down 0.3% YoY and flat QoQ).\n 5. Net Income: GAAP net income was $18 million (down 62% YoY and 85% QoQ), non-GAAP net income was $3 million (down 67% YoY and 88% QoQ).\n 6. EPS: GAAP EPS was $0.05 per share (down 62% YoY and 85% QoQ), non-GAAP EPS was $0.01 per share (down 67% YoY and 88% QoQ)."
3,"If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, what is the cost for a household that generates 30 pounds per week?","generations=[[Generation(text='1. The cost for each pound of waste generated by the household is $0.05.\n 2. The household generates 30 pounds of waste per week.\n 3. Therefore, the total weekly cost for this household would be $0.05 x 30 = $1.50.\n 4. If you want to

### Similarity Scores for Ground Truths and Answers

In [ ]:
! pip install spacy

In [85]:
import subprocess

# Command to download the Spacy model
command = "python -m spacy download en_core_web_md"

# Run the command
try:
    # This will display the output in real-time
    subprocess.run(command, check=True, shell=True)
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")

In [86]:
import spacy
import pandas as pd

# Load Spacy model
nlp = spacy.load("en_core_web_md")

all_answers = [str(answer) for answer in all_answers]
ground_truths = [str(truth) for truth in ground_truths]

# Function to compute similarity
def compute_similarity(answer, truth):
    doc1 = nlp(answer)
    doc2 = nlp(truth)
    return doc1.similarity(doc2)

# Calculate similarities for each pair
similarities = [compute_similarity(answer, truth) for answer, truth in zip(all_answers, ground_truths)]

# Create DataFrame with similarities
df_similarities = pd.DataFrame({
    'Similarity': similarities
})

# Output the DataFrame
df_similarities

,Similarity
0,0.741415
1,0.861723
2,0.827812
3,0.848423
4,0.626359
5,0.818056
6,0.907854
7,0.916571
8,0.686615
9,0.884089


### Calculating the BLEU Score

In [ ]:
! pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

import pandas as pd
df = pd.read_csv('/content/generated_answers_with_ground_truths.csv')
df

,Question,Generated Answer,Ground Truth
0,A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?,generations=[[Generation(text='500 tons x 80% = 400 tons\n The company should aim to recover 400 tons of plastic packaging annually to meet the EPR requirement.')]] llm_output=None run=[RunInfo(run_id=UUID('870d15a9-ba2f-4a5c-ac2d-1de2fb8efdc6'))],"EPR Recovery Target Calculation:\n To meet the recovery target of 80%, the company should aim to recover 80% of 500 tons, which is:\n Recovery Target=80%×500 tons = 0.80×500 tons=400 tons\n So, the company should aim to recover 400 tons of plastic packaging each year."
1,What is the report quarter and when did it end?,"generations=[[Generation(text='1. The report quarter refers to the first quarter of a fiscal year.\n 2. Based on the context information provided, the report quarter is the first quarter of Fiscal 2023.\n 3. The context information also states that the first quarter of Fiscal 2023 ended on April 29, 2023.')]] llm_output=None run=[RunInfo(run_id=UUID('8198e219-c006-4364-a611-7c2c41108405'))]","The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"
2,"What are the results for the reporting quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, op margin, net income, and EPS. Include both GAAP and non-GAAP numbers. Please also include quarter-over-quarter changes.","generations=[[Generation(text='---------------------\n • Revenue: Net sales for the reporting quarter were $3.28 billion , down 6% compared to last year.\n - Quarter-over-quarter change: Down 6%.\n • Gross margin: For the reporting quarter, gross margin was 37.9%, a decrease of 1.4 percentage points compared to last year.\n - Quarter-over-quarter change: No significant change.\n • Operating expenses (opex): Opex for the reporting quarter was $1.26 billion , down 5% compared to last year.\n - Quarter-over-quarter change: Down 5%.\n • Operating margin (op margin): Operating margin for the reporting quarter was 10.8%, a decrease of 1.7 percentage points compared to last year.\n - Quarter-over-quarter change: Down 1.7 percentage points.\n • Net income: For the reporting quarter, net income was $235 million , down 28% compared to last year.\n - Quarter-over-quarter change: Down 28%.\n • Earnings per share')]] llm_output=None run=[RunInfo(run_id=UUID('45f870a9-8f14-424d-b308-23d6c7ff5453'))]","The Results for the reporting quarter are:\n 1. Revenue: GAAP revenue was $1,214 million (up 3% YoY and 2% QoQ), while non-GAAP revenue stood at $1,200 million (up 3% YoY and 2% QoQ).\n2. Gross Margin: GAAP gross margin was 37.1% (down 0.3% YoY and flat QoQ), non-GAAP was 36.6% (down 0.4% YoY and flat QoQ).\n 3. Operating Expenses (Opex): GAAP operating expenses were $714 million (up 2% YoY and 1% QoQ), non-GAAP expenses were $687 million (up 2% YoY and 1% QoQ).\n 4. Operating Margin: GAAP operating margin came in at 13.1% (down 0.2% YoY and flat QoQ), and non-GAAP was 12.5% (down 0.3% YoY and flat QoQ).\n 5. Net Income: GAAP net income was $18 million (down 62% YoY and 85% QoQ), non-GAAP net income was $3 million (down 67% YoY and 88% QoQ).\n 6. EPS: GAAP EPS was $0.05 per share (down 62% YoY and 85% QoQ), non-GAAP EPS was $0.01 per share (down 67% YoY and 88% QoQ)."
3,"If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, what is the cost for a household that generates 30 pounds per week?","generations=[[Generation(text='1. The cost for each pound of waste generated by the household is $0.05.\n 2. The household generates 30 pounds of waste per week.\n 3. Therefore, the total weekly cost for this household would be $0.05 x 30 = $1.50.\n 4. If you want to

In [101]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Load the CSV data
df = pd.read_csv('/content/generated_answers_with_ground_truths.csv')

generated_questions = [
    [
        "If a company produces 500 tons of plastic annually and is part of an EPR scheme with an 80% recovery goal, how much plastic should it recover each year?",
        "For a company under an EPR scheme with an 80% target and an annual production of 500 tons of plastic, what is the required recovery amount?",
        "What is the annual recovery target in tons for a company producing 500 tons of plastic under an 80% EPR scheme?",
        "How many tons of plastic must a company recycle annually if it produces 500 tons and is subject to an 80% recovery EPR scheme?"
    ],
    [
        "What quarter is being reported on and when did it conclude?",
        "Identify the reporting quarter and its end date.",
        "Which fiscal quarter does the report cover and on what date did it end?",
        "Can you specify the reported quarter and its closing date?"
    ],
    [
        "List the financial results for the reported quarter in the order of revenue, gross margin, operating expenses, operating margin, net income, and EPS, including both GAAP and non-GAAP figures and their changes from the previous quarter.",
        "Provide the financial outcomes for the quarter, detailing revenue, gross margin, operating expenses, operating margin, net income, and EPS in both GAAP and non-GAAP terms, along with the changes from the last quarter.",
        "Summarize the quarter's financials in the sequence of revenue, gross margin, operating expenses, operating margin, net income, and EPS, showing both GAAP and non-GAAP values and the quarter-over-quarter variations.",
        "Detail the financial results for the quarter, focusing on revenue, gross margin, operating expenses, operating margin, net income, and EPS, including comparisons of GAAP and non-GAAP figures to the previous quarter."
    ],
    [
        "What would be the weekly cost for a household producing 30 pounds of waste under a pay-as-you-throw program charging $0.05 per pound?",
        "Calculate the weekly expense for a household disposing of 30 pounds of waste weekly with a fee of $0.05 per pound under a pay-as-you-throw scheme.",
        "Under a pay-as-you-throw system charging $0.05 per pound, how much would a household pay weekly for 30 pounds of waste?",
        "If a pay-as-you-throw program costs $0.05 per pound, what is the weekly cost for a household generating 30 pounds of waste?"
    ],
    [
        "What is the operating margin for the most recent quarter?",
        "Can you provide the operating margin figure for the latest quarter?",
        "What was the operating margin in the latest financial quarter?",
        "Determine the operating margin for the current quarter."
    ],
    [
        "Outline a risk assessment framework for EPR implementation, including probability and impact of risks like regulatory changes and market volatility, and suggest strategies to mitigate these risks.",
        "Create a risk management plan for EPR programs, assessing the likelihood and impact of risks such as regulatory shifts and technological disruptions, and recommend mitigation tactics.",
        "Develop a framework to assess and manage risks in EPR implementation, focusing on regulatory, market, and technological factors, and propose strategies to reduce their negative impacts.",
        "Construct a risk evaluation and mitigation strategy for implementing EPR, considering risks like regulatory changes and market instability, and offer solutions to minimize these risks."
    ],
    [
        "What were the net sales of Gap Inc. in the first quarter of fiscal 2023, and how did this compare to the same period last year?",
        "Report on Gap Inc.'s net sales for the first fiscal quarter of 2023 and their year-over-year comparison.",
        "How much did Gap Inc. make in net sales during the first quarter of fiscal 2023, and what was the difference from the previous year?",
        "Detail Gap Inc.'s net sales for Q1 of fiscal 2023 and compare them to the net sales from the first quarter of the previous year."
    ],
    [
        "How did the merchandise margin for Gap Inc. change compared to last year, and what influenced this change?",
        "What was the year-over-year change in merchandise margin for Gap Inc., and what were the main factors driving this change?",
        "Describe the change in Gap Inc.'s merchandise margin from the previous year and identify the key drivers of this change.",
        "Explain how Gap Inc.'s merchandise margin shifted from the previous year and the factors that contributed to this shift."
    ],
    [
        "Did Gap Inc. report an operating profit or loss for the period, and what was the amount?",
        "What was Gap Inc.'s operating result, profit or loss, for the period and the specific figure?",
        "For the reported period, did Gap Inc. have an operating profit or a loss, and what was the total?",
        "State whether Gap Inc. had an operating profit or loss during the period and provide the exact figure."
    ],
    [
        "What were the primary results for each of Gap Inc.'s brands including Old Navy, Gap, Banana Republic, and Athleta?",
        "Summarize the key financial outcomes for each of Gap Inc.'s brands: Old Navy, Gap, Banana Republic, and Athleta.",
        "Detail the significant results for Gap Inc.'s brands—Old Navy, Gap, Banana Republic, and Athleta.",
        "Report on the main financial results for each of Gap Inc.'s brands: Old Navy, Gap, Banana Republic, and Athleta."
    ],
    [
        "What are Gap Inc.'s sales projections for the second quarter and the entire fiscal year of 2023?",
        "What is Gap Inc.'s forecast for net sales in Q2 and for the full fiscal year 2023?",
        "How does Gap Inc. anticipate its net sales to perform in the second quarter and throughout fiscal year 2023?",
        "What sales expectations does Gap Inc. have for the second quarter and for the entirety of fiscal year 2023?"
    ],
    [
        "How much cash and equivalents did Gap Inc. possess at the end of the recent quarter?",
        "What was the total amount of cash and cash equivalents held by Gap Inc. at the quarter's end?",
        "At the close of the quarter, what was the balance of cash and cash equivalents for Gap Inc.?",
        "What amount of cash and cash equivalents did Gap Inc. report at the conclusion of the quarter?"
    ],
    [
        "How is Gap Inc. expecting its gross margin to vary in the second quarter and throughout the full year compared to 2022?",
        "What are Gap Inc.'s expectations for changes in gross margin for Q2 and the full year relative to 2022?",
        "In what way does Gap Inc. foresee its gross margin changing in the second quarter and over the full year compared to last year?",
        "What projections does Gap Inc. have regarding the change in gross margin for the second quarter and the entire year as compared to 2022?"
    ],
    [
        "Did Gap Inc. issue a dividend in the first quarter, and what are their future plans regarding dividends?",
        "Was there a dividend paid by Gap Inc. in the first quarter, and what is their strategy for dividends moving forward?",
        "In the first quarter, did Gap Inc. distribute dividends, and what is their dividend policy for the future?",
        "Has Gap Inc. paid out a dividend in the first quarter, and what does their dividend strategy look like going forward?"
    ],
    [
        "What strategic actions is Gap Inc. taking to implement its business plan?",
        "What are the key steps Gap Inc. is undertaking to fulfill its strategic plan?",
        "Identify the main actions being taken by Gap Inc. to execute its business strategy.",
        "What are the principal measures Gap Inc. is implementing to carry out its strategic objectives?"
    ]
]

# Add the generated questions as a new column
df['generated_questions'] = generated_questions

# Display the updated DataFrame
df

# Optionally, save the updated DataFrame to a new CSV file
df.to_csv('/content/generated_answers_with_ground_truths_updated.csv', index=False)

In [102]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize

# Each ground truth can be a list of possible correct sentences
references = [[word_tokenize(gt.lower())] for gt in df['Ground Truth']]
candidates = [word_tokenize(answer.lower()) for answer in df['Generated Answer']]

# Calculate the corpus BLEU score
score = corpus_bleu(references, candidates)

print("Corpus BLEU Score:", score)

Corpus BLEU Score: 0.062111431805526404



RAG Metrics: Answer Relevance Answer Correctness

In [ ]:
import spacy
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import string

# Load spaCy model for NLP tasks
nlp = spacy.load("en_core_web_sm")

# Load model and tokenizer
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [117]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_md")

# Initialize a SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to compute cosine similarity with SentenceTransformer
def calculate_similarity(embedding1, embedding2):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

# Function to get embeddings with SentenceTransformer
def get_embedding(text):
    return model.encode(text, convert_to_tensor=True)

# Function to compute similarity using spaCy
def compute_spacy_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Calculate Answer Correctness (comparing generated answer to ground truth)
def calculate_answer_correctness(row):
    generated_answer_emb = get_embedding(row['Generated Answer'])
    ground_truth_emb = get_embedding(row['Ground Truth'])
    return calculate_similarity(generated_answer_emb, ground_truth_emb)

# Calculate Answer Relevance (comparing the question to generated questions)
def calculate_answer_relevance(row):
    question_emb = get_embedding(row['Question'])
    generated_questions_embs = [get_embedding(q) for q in eval(row['generated_questions'])]

    relevancies = [
        calculate_similarity(question_emb, gq_emb)
        for gq_emb in generated_questions_embs
    ]
    return sum(relevancies) / len(relevancies) if relevancies else 0

# Load the CSV file
df = pd.read_csv('/content/generated_answers_with_ground_truths_updated.csv')

# Add new columns for Answer Correctness and Answer Relevance
df['Answer Correctness'] = df.apply(calculate_answer_correctness, axis=1)
df['Answer Relevance'] = df.apply(calculate_answer_relevance, axis=1)

# Display the updated DataFrame with the computed metrics
df[['Answer Relevance', 'Answer Correctness']]

,Answer Relevance,Answer Correctness
0,0.833805,0.592421
1,0.843179,0.435476
2,0.777213,0.595821
3,0.821215,0.551903
4,0.813016,0.366670
5,0.888196,0.615464
6,0.908329,0.472725
7,0.776303,0.538479
8,0.919771,0.344517
9,0.919932,0.645668


### ROGUE Scores

In [ ]:
! pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=164a15381e4bf3c1fda532edc51258cb073fbf52b140914c2bb6fb470cc6e0ff
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [118]:
from rouge_score import rouge_scorer

# Create a ROUGE scorer instance
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores for each pair of Ground Truth and Generated Answer
rouge_scores = [scorer.score(gt, ra) for gt, ra in zip(df['Ground Truth'], df['Generated Answer'])]

# Convert the list of scores to a DataFrame
rouge_scores_df = pd.DataFrame({
    "ROUGE-1": [scores['rouge1'].fmeasure for scores in rouge_scores],
    "ROUGE-2": [scores['rouge2'].fmeasure for scores in rouge_scores],
    "ROUGE-L": [scores['rougeL'].fmeasure for scores in rouge_scores]
})

rouge_scores_df.describe()  # Provide summary statistics of the scores

,ROUGE-1,ROUGE-2,ROUGE-L
count,15.000000,15.000000,15.000000
mean,0.275532,0.140926,0.209816
std,0.101132,0.092648,0.094239
min,0.134921,0.040000,0.079365
25%,0.191279,0.070997,0.137218
50%,0.263158,0.100000,0.197531
75%,0.330461,0.201250,0.236065
max,0.454545,0.325581,0.385542


In [119]:
rouge_scores_df

,ROUGE-1,ROUGE-2,ROUGE-L
0,0.454545,0.325581,0.363636
1,0.385542,0.246914,0.385542
2,0.280702,0.100000,0.192982
3,0.315789,0.160714,0.210526
4,0.171429,0.088235,0.142857
5,0.263158,0.060606,0.127820
6,0.443114,0.290909,0.359281
7,0.134921,0.040000,0.079365
8,0.184211,0.054054,0.131579
9,0.345133,0.107143,0.212389
